# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more manageable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
Successfully complete all these objectives to earn full credit. 

**Successful completion is defined as passing all the unit tests in each objective.**  

Each unit test that you pass is 1 point. 

There are 5 total possible points in this sprint challenge. 


There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

____

# Before you submit your notebook you must first

1) Restart your notebook's Kernel

2) Run all cells sequentially, from top to bottom, so that cell numbers are sequential numbers (i.e. 1,2,3,4,5...)
- Easiest way to do this is to click on the **Cell** tab at the top of your notebook and select **Run All** from the drop down menu. 

3) Comment out the cell that generates a pyLDAvis visual in objective 4 (see instructions in that section). 
____



### Import Data

In [1]:
import pandas as pd

# Load reviews from URL
data_url = 'https://raw.githubusercontent.com/LambdaSchool/data-science-practice-datasets/main/unit_4/unit1_nlp/review_sample.json'

# Import data into a DataFrame named df
# YOUR CODE HERE
df = pd.read_json(data_url, lines=True)

In [2]:
# Visible Testing
assert isinstance(df, pd.DataFrame), 'df is not a DataFrame. Did you import the data into df?'
assert df.shape[0] == 10000, 'DataFrame df has the wrong number of rows.'

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
# Optional: Consider using spaCy in your function. The spaCy library can be imported by running this cell.
# A pre-trained model (en_core_web_sm) has been made available to you in the CodeGrade container.
# If you DON'T need use the en_core_web_sm model, you can comment it out below.
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [5]:
def tokenize(doc):

  tokens = []
  for token in nlp(doc):
    if (token.is_stop == False) & (token.is_punct == False) & (token.is_space == False) & (len(token.lemma_) > 2):
      tokens.append(token.lemma_.lower())
  return tokens

In [6]:
'''Testing'''
assert isinstance(tokenize(df.sample(n=1)["text"].iloc[0]), list), "Make sure your tokenizer function accepts a single document and returns a list of tokens!"

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews (i.e. create a doc-term matrix).
2. Write a fake review and query for the 10 most similar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, use `NearestNeighbors` model for this. 

In [7]:
%%time
# Create a vector representation of the reviews 
# Name that doc-term matrix "dtm"
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words='english', tokenizer=tokenize, ngram_range=(1,2), max_df=0.9, min_df=0.1)

dtm = vect.fit_transform(df['text'])

dtm = pd.DataFrame(dtm.todense(), columns = vect.get_feature_names())

CPU times: user 4min 8s, sys: 2.29 s, total: 4min 10s
Wall time: 4min 18s


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
dtm

,amazing,ask,bad,come,day,definitely,delicious,eat,experience,food,friendly,good,great,know,like,little,look,love,need,nice,order,people,place,price,recommend,restaurant,say,service,staff,tell,thing,think,time,try,wait,want,way,work
0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.343826,0.0,0.424153,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.484419,0.000000,0.000000,0.000000,0.0,0.000000,0.328384,0.390375,0.454959,0.0,0.0,0.000000
1,0.0,0.0,0.000000,0.409838,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.536762,0.334525,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.389451,0.529472,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.188541,0.0,0.232589,0.000000,0.768829,0.47401,0.0,0.000000,0.000000,0.000000,0.0,0.249289,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.180073,0.000000,0.000000,0.0,0.0,0.000000
3,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.446125,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.755670,0.479516,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
4,0.0,0.0,0.445987,0.000000,0.000000,0.0,0.0,0.421084,0.0,0.281544,0.000000,0.253973,0.000000,0.0,0.000000,0.0,0.388179,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.392624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.418543,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.329493,0.000000,0.000000,0.000000,0.0,0.368254,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.518835,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.487282,0.0,0.0,0.499188
9996,0.0,0.0,0.000000,0.397121,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.648289,0.374237,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.531001,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
9997,0.0,0.0,0.000000,0.389218,0.000000,0.0,0.0,0.000000,0.0,0.704364,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.485570,0.000000,0.000000,0.00000,0.0,0.000000,0.341471,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
9998,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.875724,0.0,0.000000,0.0,0.000000,0.275712,0.000000,0.00000,0.0,0.000000,0.000000,0.293149,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.266749,0.000000,0.0,0.0,0.000000


In [9]:
# Create and fit a NearestNeighbors model named "nn"
from sklearn.neighbors import NearestNeighbors

# YOUR CODE HERE
nn = NearestNeighbors(n_neighbors=10)
nn.fit(dtm)

NearestNeighbors(n_neighbors=10)

In [10]:
'''Testing.'''
assert nn.__module__ == 'sklearn.neighbors._unsupervised', ' nn is not a NearestNeighbors instance.'
assert nn.n_neighbors == 10, 'nn has the wrong value for n_neighbors'

In [11]:
# Create a fake review and find the 10 most similar reviews
fake_review = [" It was the BEST!!!!! of the best attention and experience ever, i recomend this place 1000% my family and i had an amaizing time and the service was great!!!!, averyone should come here at least once in life, all people was good and happy"]
fake_review_sparse = vect.transform(fake_review)
# YOUR CODE HERE
nn_dist, nn_indexes = nn.kneighbors(fake_review_sparse.todense())

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [12]:
df['text'][nn_indexes[0][0]], nn_dist[0][0]

('Mon amie gabi is super fancy and the service is good the free bread was beyond your average olive garden.  Worth it just for the conservatory seating experience. All in all good people good times and the steak rocks.',
 0.4427737068037744)

In [13]:
fake_review[0]

' It was the BEST!!!!! of the best attention and experience ever, i recomend this place 1000% my family and i had an amaizing time and the service was great!!!!, averyone should come here at least once in life, all people was good and happy'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier.
    - Use that pipeline to train a model to predict the `stars` feature (i.e. the labels). 
    - Use that Pipeline to predict a star rating for your fake review from Part 2. 



2. Create a parameter dict including `one parameter for the vectorizer` and `one parameter for the model`. 
    - Include 2 possible values for each parameter
    - **Use `n_jobs` = 1** 
    - Due to limited computational resources on CodeGrader `DO NOT INCLUDE ADDITIONAL PARAMETERS OR VALUES PLEASE.`
    
    
3. Train the entire pipeline with a GridSearch
    - Name your GridSearch object as `gs`

In [14]:
import re
def clean_text(doc):
  doc = doc.lower()
  doc = re.sub('[^a-z]+', ' ', doc)
  return doc.strip()

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

X = df['text'].apply(clean_text)
y = df['stars']

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.9)
clf = KNeighborsClassifier(algorithm ='ball_tree')

pipe = Pipeline([
                 ('vect', vect),
                 ('clf', clf)
                 ])

params = {
    'vect__max_features':[1000],
    'clf__leaf_size':[35]
}

# Name the gridsearch instance "gs"
gs = GridSearchCV(pipe, params, cv=2, n_jobs=-1, verbose=1)
                 

gs.fit(X, y)

gs.best_score_

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_base.py:504: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


0.4854

In [16]:
gs.best_params_

{'clf__leaf_size': 35, 'vect__max_features': 1000}

In [17]:
gs.predict(fake_review)

array([5])

In [18]:
# Visible Testing
prediction = gs.predict(["I wish dogs knew how to speak English."])[0]
assert prediction in df.stars.values, 'You gs object should be able to accept raw text within a list. Did you include a vectorizer in your pipeline?'

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Set num_topics to `5`
    - Name your LDA model `lda`
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

When you instantiate your LDA model, it should look like this: 

```python
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )

```

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

## Note about  pyLDAvis

**pyLDAvis** is the Topic modeling package that we used in class to visualize the topics that LDA generates for us.

You are welcomed to use pyLDAvis if you'd like for your visualization. However, **you MUST comment out the code that imports the package and the cell that generates the visualization before you submit your notebook to CodeGrade.** 

Although you should leave the print out of the visualization for graders to see (i.e. comment out the cell after you run it to create the viz). 

In [19]:
from gensim import corpora
# Due to limited computationalresources on CodeGrader, use the non-multicore version of LDA 
from gensim.models.ldamodel import LdaModel
import gensim
import re

### 1. Estimate a LDA topic model of the review tex

In [20]:
# Remember to read the LDA docs for more information on the various class attirbutes and methods available to you
# in the LDA model: https://radimrehurek.com/gensim/models/ldamodel.html

# don't change this value 
num_topics = 5

# use tokenize function you created earlier to create tokens 
tokens = df['text'].apply(tokenize)
# create a id2word object (hint: use corpora.Dictionary)
id2word = corpora.Dictionary(tokens)
# create a corpus object (hint: id2word.doc2bow)
corpus = [id2word.doc2bow(doc_tokens) for doc_tokens in tokens]
# instantiate an lda model

lda = LdaModel(corpus=corpus,
               id2word=id2word,
               num_topics=5,
               random_state=42,
               passes=1)

#### Testing

In [21]:
# Visible Testing
assert lda.get_topics().shape[0] == 5, 'Did your model complete its training? Did you set num_topics to 5?'

#### 2. Create 1-2 visualizations of the results

In [22]:
# !pip install pyLDAvis==2.1.2

In [23]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# # Use pyLDAvis (or a ploting tool of your choice) to visualize your results 

# import pyLDAvis.gensim
# # Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
# vis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.044363 -0.014928       1        1  32.200687
1     -0.146230  0.014115       2        1  28.587732
2      0.099133 -0.065177       3        1  21.054543
3      0.001194 -0.021401       4        1  10.725058
4      0.090266  0.087391       5        1   7.431980, topic_info=         Term         Freq        Total Category  logprob  loglift
97       room  1045.000000  1045.000000  Default  30.0000  30.0000
722       car   713.000000   713.000000  Default  29.0000  29.0000
136      food  4799.000000  4799.000000  Default  28.0000  28.0000
35       time  4151.000000  4151.000000  Default  27.0000  27.0000
464      tell  1462.000000  1462.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
123     place   139.644079  5149.958810   Topic5  -5.5300  -1.0083
464      tell    99.102049  1462.209326   Topic5  -5.8729  -0.0922
41       good   121.902592  6274.434508   Topic5  -5.6659  -1.3416
400  customer    87.627938  1095.013965   Topic5  -5.9960   0.0740
136      food    94.637068  4799.613268   Topic5  -5.9190  -1.3269

[473 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
4294       5  0.932867      2006
9365       2  0.928227      4.50
3063       3  0.942838      a.m.
4301       4  0.918399    accent
6373       1  0.057577      alex
...      ...       ...       ...
7453       3  0.917320      yoga
7453       4  0.053960      yoga
11347      5  0.980563       yuk
1815       1  0.073803  zucchini
1815       2  0.910237  zucchini

[1156 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

In [24]:
words = [re.findall(r'"([^"]*)"', t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]

for id, t in enumerate(topics):
  print(f"----- Topic {id} -----")
  print(t, end="\n\n")


----- Topic 0 -----
food good place great order

----- Topic 1 -----
good place food like great

----- Topic 2 -----
time work like service tell

----- Topic 3 -----
time place come great good

----- Topic 4 -----
room hotel get service come



#### 3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model